In [5]:
instructions = open('input1.txt', 'r').read()
floor = 0
hit_basement = False
first_basement = 0
for index, instruction in enumerate(instructions):
    if instruction == '(':
        floor += 1
    else:
        floor -= 1
    if floor < 0 and not hit_basement:
        first_basement = index+1
        hit_basement = True
print(f'Santa ends up on floor {floor}')
print(f'He first hits the basement on instruction {first_basement}')

Santa ends up on floor 74
He first hits the basement on instruction 1795


In [17]:
boxes = [[int(x) for x in line.split('x')] for line in open('input2.txt', 'r').readlines()]
paper_area = 2 * sum(x*y + x*z + y*z + 0.5*min(x*y, x*z, y*z) for x,y,z in boxes)
print(f'The elves need {paper_area} square feet of wrapping paper')

ribbon = sum(min(2*(x+y), 2*(x+z), 2*(y+z)) + x*y*z for x,y,z in boxes)
print(f'The elves need {ribbon} feet of ribbon')

The elves need 1606483.0 square feet of wrapping paper
The elves need 3842356 feet of ribbon


In [28]:
instructions = open('input3.txt', 'r').read()
visited = set()
doubles = set()

first_santa = (0,0)
normal_santa = (0,0)
robo_santa = (0,0)

first_visited = set()
second_visited = set()

first_visited.add(first_santa)
second_visited.add(robo_santa)

def move(coordinate, instruction):
    x, y = coordinate
    if instruction == '>':
        return (x+1, y)
    elif instruction == '<':
        return (x-1, y)
    elif instruction == 'v':
        return (x, y-1)
    elif instruction == '^':
        return (x, y+1)

for index, instruction in enumerate(instructions):
    first_santa = move(first_santa, instruction)
    first_visited.add(first_santa)
    
    if index % 2 == 0:
        normal_santa = move(normal_santa, instruction)
        second_visited.add(normal_santa)
    else:
        robo_santa = move(robo_santa, instruction)
        second_visited.add(robo_santa)

print(f'Santa has visited {len(first_visited)} houses')
print(f'Santa and Robo-Santa have visited {len(second_visited)} houses')
    

Santa has visited 2592 houses
Santa and Robo-Santa have visited 2360 houses


In [ ]:
import hashlib

input = 'yzbqklnj'
counter = 0

five_digits = 1e1000
six_digits = 1e1000

while True:
    m = hashlib.md5()
    m.update(f'{input}{counter}'.encode('utf-8'))
    if m.hexdigest().startswith('00000'):
        five_digits = counter if counter < five_digits else five_digits
    if m.hexdigest().startswith('000000'):
        six_digits = counter if counter < six_digits else six_digits
        break
    counter += 1
print(f'The lowest integer to produce a hash starting with five zeros is {five_digits}')
print(f'The lowest integer to produce a hash starting with six zeros is {six_digits}')